In [1]:
import numpy as np

class Camera:
    def __init__(self, e, t, g, frustum) -> None:
        self.e = e
        self.t = t 
        self.g = g
        self.frustum = frustum # [t,b,l,r,n,f]
        
        # M_view     
        self.T = np.array([ [1,0,0,-self.e[0]],
                            [0,1,0,-self.e[1]],
                            [0,0,1,-self.e[2]],
                            [0,0,0,1]])
                      
        gt = np.cross(self.g, self.t)
        self.R = np.array([ [gt[0],gt[1],gt[2],0],
                            [self.t[0],self.t[1],self.t[2],0],
                            [-self.g[0],self.g[1],-self.g[2],0],
                            [0,0,0,1]]) 
        self.view = np.dot(self.R, self.T)
    
        # M_per
        t,b,l,r,n,f = self.frustum[0], self.frustum[1], self.frustum[2], self.frustum[3], self.frustum[4], self.frustum[5]
        self.Persp = np.array([ [2*n/(r-l),0,(l+r)/(l-r),0],
                                [0,2*n/(t-b),(b+t)/(b-t),0],
                                [0,0,(f+n)/(n-f),2*f*n/(f-n)],
                                [0,0,1,0]])

![image](./figure/1-2.png)

In [5]:
myCamera = Camera(e=[1,0,0], g=[0,0,-1], t=[0,1,0], frustum=[1,-1,-1,1,-2,-6])
triangle_1 = {"A": np.array([0,-1,-4,1]), "B": np.array([2,-1,-4,1]), "C": np.array([1,1,-6,1])}
triangle_2 = {"A": np.array([0,-1,-4,1]), "B": np.array([1,0,-3,1]), "C": np.array([1,1,-6,1])}
object_ls = [triangle_1, triangle_2]
# triangle texture
Point_light = np.array([0,2,0])

# Shading

In [20]:
# Screen
H = 100
W = 100
screen = np.zeros([H, W, 3])

def ViewScreen(triangle_ndc, H, W): # [H, W] is the screen resolution
    triangle_screen = np.zeros([3, 2])
    for i, node in enumerate(["A","B","C"]):
        triangle_screen[i][0] = (0.5*triangle_ndc[node][0]+0.5)*W
        triangle_screen[i][1] = (0.5*triangle_ndc[node][1]+0.5)*H
    return triangle_screen
def whether_in_triangle(triangle_screen, Q):
    # whether coodinate Q is in the triangle?
    # AB×AQ, BC×BQ, CA×CQ point to same side of the screen(sign(z) are same) ->  Q in triangle
    A = np.append(triangle_screen[0], [0], axis=0) 
    B = np.append(triangle_screen[1], [0], axis=0)
    C = np.append(triangle_screen[2], [0], axis=0)
    Q = np.append(Q, [0], axis=0)
    x1 = np.cross(B-A, Q-A)
    x2 = np.cross(C-B, Q-B)
    x3 = np.cross(A-C, Q-C)
    if np.sign(x1[2]) == np.sign(x2[2]) == np.sign(x3[2]):
        return True
    else:
        return False

# Z_buffer

pixel-level depth information

We only know the z for the points of the triangle.
- use barycentric coordinate for each pixel in the triangle

# Barycentric coordinate

Interpolation of the **attributes**, including:
- texture
- z-buffer
- ...


In [40]:
def BaryCentric(A, B, C, P):
    # return the bary-coordinate of P for ABC
    S_ABC = np.linalg.norm(np.cross(B-A, C-A))/2
    S_BCP = np.linalg.norm(np.cross(C-B, P-B))/2
    u = S_BCP/S_ABC
    S_ACP = np.linalg.norm(np.cross(C-A, P-A))/2
    v = S_ACP/S_ABC
    # S_ABP = np.linalg.norm(np.cross(B-A, P-A))/2
    w = 1-u-v
    if u>0 and v>0 and 1-u-v>0:
        return u, v, w
    else:
        print("P is not in ABC")
        return None
# ABCQ in screen space (z==0)
def BaryCentric_2d(A, B, C, P):
    # A = [u,v]
    # return the bary-coordinate of P for ABC
    A = np.array([A[0], A[1], 0])
    B = np.array([B[0], B[1], 0])
    C = np.array([C[0], C[1], 0])
    P = np.array([P[0], P[1], 0])

    S_ABC = np.linalg.norm(np.cross(B-A, C-A))/2
    S_BCP = np.linalg.norm(np.cross(C-B, P-B))/2
    u = S_BCP/S_ABC
    S_ACP = np.linalg.norm(np.cross(C-A, P-A))/2
    v = S_ACP/S_ABC
    # S_ABP = np.linalg.norm(np.cross(B-A, P-A))/2
    w = 1-u-v
    if u>0 and v>0 and 1-u-v>0:
        return u, v, w
    else:
        print("P is not in ABC")
        return None

In [75]:
triangle_1 = {"A": np.array([0,-1,-4,1]), "B": np.array([2,-1,-4,1]), "C": np.array([1,1,-6,1]), 
              "ka": np.array([1,0,0]), "kd": np.array([1,0,0]), "ks": np.array([1,1,1])} # red
triangle_2 = {"A": np.array([0,-1,-4,1]), "B": np.array([1,0,-3,1]), "C": np.array([1,1,-6,1]),
              "ka": np.array([0,1,0]), "kd": np.array([0,1,0]), "ks": np.array([1,1,1])} # green
object_ls = [triangle_1, triangle_2]
# triangle texture
# white light
mylight = {"pos": np.array([0,2,0]), "ka": np.array([1,1,1]), "kd": np.array([1,1,1]), "ks": np.array([1,1,1]), "Ia": 0.1, "I": 0.9}

In [57]:
def homo_to_xyz(a):
    a = np.array([a[0]/a[3], a[1]/a[3], a[2]/a[3], a[3]/a[3]])
    return a
def inverse_dir(a):
    for i in range(a.shape[0]):
        a[i] = -a[i]
    return a
def normalize(a):
    for i in range(a.shape[0]):
        a[i] = a[i]/np.linalg.norm(a)
    return a


# Shading
Blinn-Phong model.

![image](./figure/1-4.1.png)
![image](./figure/1-3.png)

In [84]:
import math
def Shading(light, triangle):
    A, B, C = triangle["A"][:3], triangle["B"][:3], triangle["C"][:3]
    L = np.zeros([3, 3]) # three nodes, RGB
    n = normalize(np.cross(B-A, B-C))
    if n[2]<0:
        n = inverse_dir(n)
    for i, id in enumerate(["A", "B", "C"]):
        coo = triangle[id][:3]
        l = normalize(light['pos']-coo)
        r2 = math.pow(np.linalg.norm(coo-light['pos']), 2)
        v = np.array([0,0,1]) # from screen to eye: z-axis
        h = normalize(v+l)
        ambient = light["ka"]*triangle["ka"]*light["Ia"]
        diffuse = light["kd"]*triangle["kd"]*(light["I"]/r2)*max(0, np.dot(n, l))
        highlight = light["ks"]*triangle["ks"]*(light["I"]/r2)*math.pow(max(0, np.dot(n, h)), 10)
        L[i] = ambient + diffuse + highlight
    return L

In [82]:
triangle_1 = {"A": np.array([0,-1,-4,1]), "B": np.array([2,-1,-4,1]), "C": np.array([1,1,-6,1]), 
              "ka": np.array([1,0,0]), "kd": np.array([1,0,0]), "ks": np.array([1,1,1])} # red
triangle_2 = {"A": np.array([0,-1,-4,1]), "B": np.array([1,0,-3,1]), "C": np.array([1,1,-6,1]),
              "ka": np.array([0,1,0]), "kd": np.array([0,1,0]), "ks": np.array([1,1,1])} # green
object_ls = [triangle_1, triangle_2]
# triangle texture
# white light
mylight = {"pos": np.array([0,2,0]), "ka": np.array([1,1,1]), "kd": np.array([1,1,1]), "ks": np.array([1,1,1]), "Ia": 0.5, "I": 0.5}

In [86]:
import imageio
from time import time


# Project object in world space to NDC space ([-1,1]^3)
for object in object_ls:
    object['ndc'] = {}
    for i in ["A","B","C"]:
        # to camera space
        triangle_c = np.dot(myCamera.view, object[i])
        # to ndc space
        object['ndc'][i] = homo_to_xyz(np.dot(myCamera.Persp, triangle_c))


# Screen info
H = 100
W = 100
screen = np.zeros([H, W, 3])

# NDC space to screen space
for object in object_ls:
    object['screen'] = ViewScreen(object['ndc'], H, W) # shape=[3,2]
    
z_buffer = np.ones([H, W])*myCamera.frustum[5]-1
for i in range(W):
    for j in range(H):
        # tranverse the object
        idx = j
        idy = H-i
        for object in object_ls:
            if whether_in_triangle(object['screen'], [idx+0.5, idy+0.5]):
                u, v, w = BaryCentric_2d(object['screen'][0], object['screen'][1], object['screen'][2], np.array([idx+0.5, idy+0.5]))
                z_inter = u*object['A'][2] + v*object['B'][2] + w*object['C'][2]
                if z_inter>z_buffer[i][j]:
                    z_buffer[i][j] = z_inter
                    shading_ABC = Shading(light=mylight, triangle=object)
                    result = u*shading_ABC[0]+v*shading_ABC[1]+w*shading_ABC[2]
                    screen[i][j] = result
                    print(result)
imageio.imsave("./result/1/2_triangle_shading.png", screen)


[0.663222 0.081611 0.081611]
[0.08253889 0.66507779 0.08253889]
[0.66293591 0.08146795 0.08146795]
[0.66395703 0.08197852 0.08197852]
[0.08569009 0.67138018 0.08569009]
[0.66367094 0.08183547 0.08183547]
[0.66497816 0.08248908 0.08248908]
[0.08434486 0.66868973 0.08434486]
[0.66469206 0.08234603 0.08234603]
[0.08884128 0.67768257 0.08884128]
[0.66440597 0.08220298 0.08220298]
[0.66411988 0.08205994 0.08205994]
[0.66571319 0.08285659 0.08285659]
[0.08749606 0.67499212 0.08749606]
[0.66542709 0.08271355 0.08271355]
[0.09199248 0.68398496 0.09199248]
[0.665141  0.0825705 0.0825705]
[0.66485491 0.08242745 0.08242745]
[0.66673431 0.08336715 0.08336715]
[0.08615083 0.67230167 0.08615083]
[0.66644822 0.08322411 0.08322411]
[0.09064725 0.68129451 0.09064725]
[0.66616212 0.08308106 0.08308106]
[0.09514367 0.69028735 0.09514367]
[0.66587603 0.08293801 0.08293801]
[0.66558994 0.08279497 0.08279497]
[0.66530384 0.08265192 0.08265192]
[0.66775543 0.08387772 0.08387772]
[0.08480561 0.66961122 0.0848

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [29]:
object['screen']

array([[25.        , 25.        ],
       [75.        , 25.        ],
       [50.        , 66.66666667]])